# Laboratórios de Bioinformática
### Análise de homologias por BLAST

**Grupo 6**: Joana Gabriel, Maria Couto, Teresa Coimbra

### Procura de genes Homólogos

A similaridade entre sequências pode ser verificada através de algoritmos como o BLAST. O BLAST(Basic Local Alignment Search Tool) possibilita  uma comparação da informação de sequências biológicas (seja de nucleótidos ou de aminoácidos) fornecidas na pesquisa, com sequências contidas numa base de dados. Os resultados desta comparação são filtrados de acordo com o grau de semelhança existente entre as sequências. 

Através do BioPython é possível executar o BLAST com os parâmetros desejados e analisar os resultados. A função utilizada foi a função qblast() que está contida no módulo Bio.Blast.NCBIWWW.
Esta função recebe três parâmetros não opcionais, que incluem o programa a utilizar, a base de dados e a sequência a pesquisar (query). 

O programa a utilizar pode ser uma das opções seguintes de pesquisa:
- nucleótidos (blastn), 
- proteínas (blastp), 
- pesquisa contra sequências existentes na base de dados do NCBI (blastx),
- pesquisa por sequências de nucleótidos traduzidas utilizando uma sequência de uma proteína (tblastn),
- pesquisa por sequências de nucleótidos traduzidas utilizando uma sequência de nucleótidos traduzida (tblastx).


Quanto à base de dados sobre a qual é efetuada a pesquisa existem diversas possibilidades, tais como: nt, nr, refseq_rna, est , swissprot, SRA, TSA, refseq_genomes, entre outras. 
A sequência a pesquisar ou query pode ser fornecida como uma string (a própria sequência), em formato FASTA ou com um identificador. 

De acordo com a pesquisa, são identificadas proteínas ou genes homólogos, que demonstram uma similaridade estatisticamente significativa (pois esta reflete uma ancestralidade em comum). 

Foram efetuados BLAST para os ficheiros FASTA gerados na pesquisa realizada anteriormente, utilizando a opção blastp. Já a base de dados selecionada foi a Swissprot. Os ficheiros gerados têm o formato xml.


In [8]:
def blast(filename,search_program = 'blastp', database = 'swissprot'):
    '''
    Esta função permite-nos invocar o blast utilizando a função qblast() do Biopython. 
    Recebe:
        -nome do ficheiro .faa (str);
        -programa do blast utilizado para fazer a pesquisa (str);
        -nome da base de dados sobre a qual é efetuada a pesquisa (str).
    Retorna:
        os ficheiros xml correspondentes com o resultado do Blast'''
    
    from Bio.Blast import NCBIWWW
    from Bio import SeqIO
    
    record        = SeqIO.read(open(filename+'.faa'), format = 'fasta')
    result_handle = NCBIWWW.qblast('blastp','swissprot',record.format('fasta')) # escolhemos a swissprot por ser uma base de dados curada
    save_file     = open(filename+'.xml','w') 
    save_file.write(result_handle.read())
    save_file.close()
    result_handle.close()

### Correr o Blast de modo interativo:

In [3]:
def run_blast():
    fn        = input('Inserir o nome do ficheiro com a sequência para correr o blast: ')
    decide    = int(input('O programa de pesquisa utilizado é o blastp e a base de dados a Swissprot. Deseja alterar estes parâmetros? 0 - Não, manter; 1 - Sim.'))
    if decide == 0:
        blast(fn)
    else:
        opt_prog = input('Escolher um programa do blast para efetuar a pesquisa: blastn, blastp, blastx, tblast, tblastx: ')
        assert opt_prog in ['blastn','blastp','blastx','tblast','tblastx'], 'O programa não consta na lista de possibilidades do blast.'
        opt_db   = input('Escolher uma base de dados sobre a qual será efetuada a pesquisa. Exemplos: nt,nr,refseq_rna,est,swissprot,SRA,TSA,refseq_genomes. Para mais opções verificar ftp://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf. Base de dados escolhida: ')
        blast(fn,opt_prog,opt_db)

In [5]:
#run_blast()

### Correr o Blast dos ficheiros obtidos na pesquisa feita anteriormente:

In [6]:
#blast('spP0DTC9.1NCAP_SARS2')     # N protein
#blast('NP_001011724.1')           # heterogeneous nuclear ribonucleoprotein
#blast('NP_001011725.1')           # heterogeneous nuclear ribonucleoprotein
#blast('NP_002127.1')              # heterogeneous nuclear ribonucleoprotein
#blast('NP_112420.1')              # heterogeneous nuclear ribonucleoprotein
#blast('NP_001138574.1')           # SMAD3       
#blast('NP_001138575.1')           # SMAD3
#blast('NP_001138576.1')           # SMAD3
#blast('NP_005893.1')              # SMAD3

### Número de alinhamentos em cada ficheiro xml resultante: 

Cada sequência gera um determinado número de alinhamentos associado.

In [2]:
def n_align(filename):
    '''Recebe o nome de um ficheiro xml(str)
    Retorna o número de alinhamentos existentes no ficheiro (int)'''
    from Bio.Blast import NCBIXML
    result_handle = open('./'+filename+'.xml') 
    blast_records = NCBIXML.read(result_handle)
    c = 0 
    for alignment in blast_records.alignments:
        c+=1
    print('O número de alinhamentos para o ficheiro',filename,'é:', c)

In [3]:
filenames = ['spP0DTC9.1NCAP_SARS2','NP_001011724.1','NP_001011725.1','NP_001138574.1','NP_001138575.1','NP_001138576.1','NP_005893.1','NP_002127.1','NP_112420.1']
for fname in filenames:
    n_align(fname)

O número de alinhamentos para o ficheiro spP0DTC9.1NCAP_SARS2 é: 50
O número de alinhamentos para o ficheiro NP_001011724.1 é: 50
O número de alinhamentos para o ficheiro NP_001011725.1 é: 50
O número de alinhamentos para o ficheiro NP_001138574.1 é: 38
O número de alinhamentos para o ficheiro NP_001138575.1 é: 38
O número de alinhamentos para o ficheiro NP_001138576.1 é: 41
O número de alinhamentos para o ficheiro NP_005893.1 é: 40
O número de alinhamentos para o ficheiro NP_002127.1 é: 50
O número de alinhamentos para o ficheiro NP_112420.1 é: 50


# Automatização da decisão de existência de homologias significativas 

### Análise dos resultados obtidos


Podemos inferir que existe homologia entre as sequências de acordo com o seu grau de semelhança. É possível que existam sequências homólogas que não partilhem um elevado grau de semelhança, principalmente quando se comparam sequências de DNA, contudo, neste caso, tendo em conta que é feita uma comparação entre sequências de aminoácidos, considera-se que este risco é menor. 

A semelhança pode ser avaliada de acordo com diferentes parâmetros tais como o E-value, a percentagem de identidade e os bit-scores. 

Para avaliar  a homologia entre sequências os parâmetros mais relevantes são o **E-value** e os **bit-scores**.

O valor do E-value depende da extensão do alinhamento e do número de sequências que existem na base de dados. Quanto menor o E-value, mais significativo o alinhamento, pois este valor traduz o número esperado de vezes que um alinhamento pode ocorrer devido ao acaso, tendo em conta os aloritmos existentes. 

A significância do bit-score depende do tamanho da base de dados e da sequência query. De uma forma geral, é possível considerar um resultado como estatisticamente significativo se o bit-score for de 40-50.

Os HSP(High-scoring Segment Pair) correspondem aos melhores alinhamentos locais obtido (sem espaçamentos).

Consideramos homologias significativas as que têm um score alto (>200).


In [4]:
def blast_result(filename, E_VALUE_THRES=0.0001, bit_score = 50, SCORE=200):
    from Bio.Blast import NCBIXML
    from Bio import SeqIO
    result_handle = open('./'+filename+'.xml') 
    blast_records = NCBIXML.read(result_handle)
    
    List1 = []
    for alignment in blast_records.alignments:
        for hsp in alignment.hsps:
            l = []
            if (hsp.expect < E_VALUE_THRES) and (hsp.bits > bit_score) and (hsp.score > SCORE):
                print('****Alignment****')
                print('sequence:', alignment.title)
                print('e.value:',hsp.expect)
                print('bit-score:',hsp.bits)
                print('hsp score:', hsp.score)
                #print ( hsp.query [0:75] + '...')             # query
                #print( hsp.match [0:75] + '...')              # alinhamento
                #print(hsp.sbjct [0:75] + '...')               # seq que alinhou
                
                l.append(hsp)
                
                #l.append(alignment.title)
                #l.append(hsp.expect)
                #l.append(hsp.bits)
                #l.append(hsp.score)
                List1.append(l)
        
    return List1

In [ ]:
def blast_result(filename, E_VALUE_THRES=0.0001, bit_score = 50, SCORE=200):
    from Bio.Blast import NCBIXML
    from Bio import SeqIO
    result_handle = open('./'+filename+'.xml') 
    blast_records = NCBIXML.read(result_handle)
    
    for alignment in blast_records.alignments:
        for hsp in alignment.hsps:
       
            if (hsp.expect < E_VALUE_THRES) and (hsp.bits > bit_score) and (hsp.score > SCORE):
                print('****Alignment****')
                print('sequence:', alignment.title)
                print('e.value:',hsp.expect)
                print('bit-score:',hsp.bits)
                print('hsp score:', hsp.score)
    
    return List1

In [8]:
blast_result('NP_001011725.1')

****Alignment****
sequence: sp|Q32P51.2| RecName: Full=Heterogeneous nuclear ribonucleoprotein A1-like 2; Short=hnRNP A1-like 2; AltName: Full=hnRNP core protein A1-like 2 [Homo sapiens]
e.value: 0.0
bit-score: 642.114
hsp score: 1655.0
****Alignment****
sequence: sp|A5A6H4.1| RecName: Full=Heterogeneous nuclear ribonucleoprotein A1; Short=hnRNP A1; AltName: Full=Helix-destabilizing protein; AltName: Full=Single-strand-binding protein; AltName: Full=hnRNP core protein A1; Contains: RecName: Full=Heterogeneous nuclear ribonucleoprotein A1, N-terminally processed [Pan troglodytes] >sp|P09867.2| RecName: Full=Heterogeneous nuclear ribonucleoprotein A1; Short=hnRNP A1; AltName: Full=Helix-destabilizing protein; AltName: Full=Single-strand RNA-binding protein; AltName: Full=Unwinding protein 1; Short=UP1; AltName: Full=hnRNP core protein A1; Contains: RecName: Full=Heterogeneous nuclear ribonucleoprotein A1, N-terminally processed [Bos taurus] >sp|P49312.2| RecName: Full=Heterogeneous nucle

[[<Bio.Blast.Record.HSP at 0x1c6caa40520>],

In [7]:
blast_result('NP_112420.1')

****Alignment****
sequence: sp|P09651.5| RecName: Full=Heterogeneous nuclear ribonucleoprotein A1; Short=hnRNP A1; AltName: Full=Helix-destabilizing protein; AltName: Full=Single-strand RNA-binding protein; AltName: Full=hnRNP core protein A1; Contains: RecName: Full=Heterogeneous nuclear ribonucleoprotein A1, N-terminally processed [Homo sapiens]
e.value: 0.0
bit-score: 721.079
hsp score: 1860.0
****Alignment****
sequence: sp|A5A6H4.1| RecName: Full=Heterogeneous nuclear ribonucleoprotein A1; Short=hnRNP A1; AltName: Full=Helix-destabilizing protein; AltName: Full=Single-strand-binding protein; AltName: Full=hnRNP core protein A1; Contains: RecName: Full=Heterogeneous nuclear ribonucleoprotein A1, N-terminally processed [Pan troglodytes] >sp|P09867.2| RecName: Full=Heterogeneous nuclear ribonucleoprotein A1; Short=hnRNP A1; AltName: Full=Helix-destabilizing protein; AltName: Full=Single-strand RNA-binding protein; AltName: Full=Unwinding protein 1; Short=UP1; AltName: Full=hnRNP core 

[[<Bio.Blast.Record.HSP at 0x1c6ca9eddc0>],

In [9]:
blast_result('NP_005893.1')

****Alignment****
sequence: sp|P84022.1| RecName: Full=Mothers against decapentaplegic homolog 3; Short=MAD homolog 3; Short=Mad3; Short=Mothers against DPP homolog 3; Short=hMAD-3; AltName: Full=JV15-2; AltName: Full=SMAD family member 3; Short=SMAD 3; Short=Smad3; Short=hSMAD3 [Homo sapiens] >sp|P84024.1| RecName: Full=Mothers against decapentaplegic homolog 3; Short=MAD homolog 3; Short=Mad3; Short=Mothers against DPP homolog 3; AltName: Full=SMAD family member 3; Short=SMAD 3; Short=Smad3 [Sus scrofa] >sp|P84025.1| RecName: Full=Mothers against decapentaplegic homolog 3; Short=MAD homolog 3; Short=Mad3; Short=Mothers against DPP homolog 3; AltName: Full=SMAD family member 3; Short=SMAD 3; Short=Smad3 [Rattus norvegicus] >sp|Q8BUN5.2| RecName: Full=Mothers against decapentaplegic homolog 3; Short=MAD homolog 3; Short=Mad3; Short=Mothers against DPP homolog 3; Short=mMad3; AltName: Full=SMAD family member 3; Short=SMAD 3; Short=Smad3 [Mus musculus]
e.value: 0.0
bit-score: 890.567
hsp

[[<Bio.Blast.Record.HSP at 0x1c6cc318a60>],

A lista de listas 'a' vai ter os objectos Blast Record HSP resultantes da pesquisa anterior

In [24]:
for i in a:
    print(type(i))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


### As sequências que resultaram em E-values baixos, bit-scores elevados e scores elevados têm uma elevada probabilidade de serem sequências homólogas. 
Assim, o resultado da função **blast_result** corresponde à lista de sequências homólogas. 